## Genetic Algorithm
The number of possibilities (the search space) generated by all the combinations of the lexicons is $2^{17} = 131,072$. That means a very big space to check for the best combination.

A genetic algorithm is going to be used for the search. [This link](https://github.com/DEAP/notebooks/blob/master/OneMax.ipynb) and [this](https://github.com/lmarti/evolutionary-computation-course/blob/master/AEC.02%20-%20Elements%20of%20Evolutionary%20Algorithms.ipynb) will be used as the base to create the algorithm.

### Definitions
For a genetic algorithm we need to define mainly:

- *individuals*: what is an individual and how the genes define them
- *population size*: the population size that will be kept constant
- *fitness function*: the fitness function that will determine wich individuals are more fitted in the problem context
- *selection process*: how will the most fitted be selected. There are many possible like the best K individuals, the more diverse ones, X% better and y% worst
- *genetic variation*: how will the individuals evolve and the chromossomes be mixed

### Simple Evolution
The main definitions for a simple model, to make sure the concept is ok.

** Individuals **
Each gene position is going to be one set of features generated by each of the lexicons. I.e.:

*Individual 1* - gene [10000000000]
This gene means he has the lexicon Bin Liu enabled

** Population Size **
Due to processing power, the population size P will be 10.

** Fitness function **
The fitness function is going to test the lexicons combinations and find the best result for the F1-Score according to the SemEval concept described previously. 

I.e: For the individual 1, the Bing Liu Lexicon's features will be created and then the best average score for a model will be used as the fitness value.

** Selection **
For each generation, the top P (population size) will be selected to the next round.

** Genetic variation **
The individuals will have their own genetic change (mutation) of 1 gene max and the crossover (mate) of 1 point


## GA Primer

In [ ]:
%matplotlib inline
import random
from deap import base, creator, tools, algorithms
import libs.resources as r
import numpy as np

In [ ]:
# Simple definition
IND_SIZE = len(r.lexs)
POP_SIZE = 30
rnd_seed = 9000

In [ ]:
def rand():
    """
    My own random that enables the gene 10% of the time
    """
    return int(random.randint(0,10) >=9)

# define an individual gene
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

# initialize the simple evolution toolbox
simple_tb = base.Toolbox()

# define the individual generator
simple_tb.register("attr_bool", rand)
simple_tb.register("individual", tools.initRepeat, creator.Individual, 
                  simple_tb.attr_bool, n=IND_SIZE)

# define the population generator
simple_tb.register("population", tools.initRepeat, list, simple_tb.individual)

In [ ]:
print 'Individual 1', simple_tb.individual()
print 'Individual 2', simple_tb.individual()

In [ ]:
# seting all the genes to 1
a = simple_tb.individual()
for i in range(len(a)):
    a[i] = 1
a

In [ ]:
pop = simple_tb.population(POP_SIZE)
print 'Population', pop

In [ ]:
reload(r)

In [ ]:
# evaluation function
def evaluation(individual):
    return sum(individual),

In [ ]:
# create the evolutionary parts
simple_tb.register("evaluate", evaluation)
simple_tb.register("mate", tools.cxOnePoint) # mate changing in one point
simple_tb.register("mutate", tools.mutFlipBit, indpb=0.20) # flip a bit with 10% chance
simple_tb.register("select", tools.selBest)

In [ ]:
# register the statistics
stats = tools.Statistics(key=lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
#stats.register("std", numpy.std)
stats.register("min", np.min)
stats.register("max", np.max)

# save the best
hof = tools.HallOfFame(3)

In [ ]:
final_pop, log = algorithms.eaSimple(pop, simple_tb, cxpb=0.5, mutpb=0.2, ngen=50,
                            stats=stats, halloffame=hof, verbose=False)

In [ ]:
def print_evolution(log):
    import matplotlib.pyplot as plt
    gen, avg, min_, max_ = log.select("gen", "avg", "min", "max")
    plt.plot(gen, avg, label="average")
    plt.plot(gen, min_, label="minimum")
    plt.plot(gen, max_, label="maximum")
    plt.xlabel("Generation")
    plt.ylabel("Fitness")
    plt.legend(loc="lower right")
#print_evolution(log)

## Applying to Lexicon Selection

In [ ]:
import libs.pipeline as pipe
import libs.resources as r
import libs.utils as u

In [ ]:
# read the base data
train_dataset_stop = pipe.load_dump_data('train_base_data.pck')
dev_dataset_stop = pipe.load_dump_data('dev_base_data.pck')
test_dataset_stop = pipe.load_dump_data('test_base_data.pck')
labels = pipe.load_dump_data('labels.pck')
dev_labels = pipe.load_dump_data('dev_labels.pck')
gold = pipe.load_dump_data('gold.pck')

In [ ]:
# checking one individual
ind = simple_tb.individual()
ind

In [ ]:
# load the lexicons according to the gens
ind_lexs = [r.lexs[idx] for idx, present in enumerate(ind) if present]
ind_lexs

In [ ]:
r.lexs

In [ ]:
def load_lex_features_dump(lex):
    """
    Load the saved trained and test lexicon features
    """
    tmp_train = pipe.load_dump_data(lex.prefix+'_train.pck')
    tmp_test = pipe.load_dump_data(lex.prefix+'_test.pck')
    
    return tmp_train, tmp_test
lex_train, lex_test = load_lex_features_dump(ind_lexs[0])
lex_train.shape, lex_test.shape

In [ ]:
def decode_gene(ind):
    """
    Decode a gene to a information that can be processed.
    """
    return [r.lexs[idx] for idx, present in enumerate(ind) if present]
    
def create_ind_features(ind, base_train, base_test):
    """
    Given an individual, create the features according to its genes
    Params:
        ind: individual generated in the genetic algorithm
    Returns:
        train and test data as a scipy sparse matrix
    """
    # load the lexicons according to the gens
    ind_lexs = decode_gene(ind)
    
    # for each of the lexicons, merge them
    final_train = base_train.copy()
    final_test = base_test.copy()
#     print final_train.shape, final_test.shape
    for lex in ind_lexs:
        lex_train, lex_test = load_lex_features_dump(lex)
#         print lex, lex_train.shape, lex_test.shape
        final_train, final_test = pipe.join_lex_features(final_train, lex_train, final_test, 
                                                    lex_test, verbose=False, create_vec=False)
        
    return final_train, final_test

In [ ]:
import sklearn.model_selection as model_selection

def fitness_function(individual):#, base_train, base_test, labels, gold):
    """
    Given an individual, make the fitness test and find a value that defines this individual
    """
    print 'Testing', individual
    
    # check if this individual was tested before
    key = '{}'.format(individual)
    prev = all_individuals.get(key, None)
    if prev:
        return prev,
    else:
        # create the feature dataset
        train_data, test_data = create_ind_features(individual, base_train, base_test)

        # run the experiment with multiple algorithms
        ret_df = pipe.run_multiple_class(skf, train_data, labels, test_data, gold, rnd_seed=rnd_seed)

        # get the best available value
        final_val = ret_df['dev score'].max()
        
        all_individuals[key] = final_val
        
        return final_val, 
#final, df = fitness_function(ind, train_dataset_stop, test_dataset_stop, labels, gold)

In [ ]:
skf = model_selection.StratifiedKFold(10, random_state=rnd_seed)
base_train = train_dataset_stop
base_test = test_dataset_stop

all_individuals = {}

## PS:
use the script to run the evolution algorithm. Jupyter notebook does not allow it to run it here.

In [ ]:
simple_tb.register("evaluate", fitness_function)
final_pop, log = algorithms.eaSimple(pop, simple_tb, cxpb=0.5, mutpb=0.2, ngen=6,
                            stats=stats, halloffame=hof, verbose=True)

In [ ]:
print_evolution(log)

In [ ]:
log

In [ ]:
hof.items

In [ ]:
all_individuals

In [ ]:
with open('../3-Output/population.csv', 'w') as f:
    results = sorted(all_individuals.iteritems(), key=lambda (k,v): v, reverse=True)
    for line in results:
        k,v = line
        f.write('{};{}\n'.format(k,v))

### Decoding a Gene
Given a Gene, we can decode it using the function below

In [ ]:
available_lex = [lex for lex in r.lexs if lex != r.emosnet]

ind1 = [0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]
ind2 = [0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]
ind3 = [1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0]
ind4 = [1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0]
[available_lex[idx] for idx, present in enumerate(ind1) if present]

In [ ]:
[available_lex[idx] for idx, present in enumerate(ind4) if present]

In [ ]:
available_lex